In [58]:
from codelin.models.const_tree import C_Tree
from codelin.encs.constituent import *
from codelin.utils.constants import *
from codelin.models.linearized_tree import LinearizedTree
from codelin.models.const_label import C_Label
from nltk.tree import Tree

import pandas as pd
import os
import copy

def pt(d, t):
    '''
    Print tree if d is True
    '''
    if d:
        Tree.fromstring(str(t)).pretty_print()

def bin_print(ts):
    '''
    Print binary tree
    '''
    Tree.fromstring(str(C_Tree.to_binary_right(C_Tree.from_string(ts)))).pretty_print()

tsa = "(NTA (PA a) (NTB (NTC (NTD (PB b) (NTE (PC c) (NTF (NTG (PD d) (PE e)) (PF f)))) (PG g)) (PH h)))"
tsb = "(NTA (PA a) (NTB (NTC (PB b) (NTF (NTD (PC c) (NTE (PD d) (PE e))) (PF f))) (NTG (NTH (PG g) (NTI (PH h) (NTJ (PI i) (PJ j)))) (PK k))))"
tsc = "(NTA (NTU (NTV (PA11 a11) (PA12 a12)) (NTW (PA21 a21) (PA22 a22))) (NTB (NTC (PB b) (NTF (NTD (PC c) (NTE (PD d) (PE e))) (PF f))) (NTG (NTH (PG g) (NTI (PH h) (NTJ (PI i) (PJ j)))) (PK k))))"
tsd = "(NTA (NTU (NTV (NTP (S s) (T t)) (PA12 a12)) (NTW (PA21 a21) (NTQ (X x) (Y y)))) (NTB (NTC (PB b) (NTF (NTD (PC c) (NTE (PD d) (PE e))) (PF f))) (NTG (NTH (PG g) (NTI (PH h) (NTJ (PI i) (PJ j)))) (PK k))))"

ts1 = "(S (S (NP (JJ Big) (NN investment) (NNS banks)) (VP (VBD refused) (S (VP (TO to) (VP (VB step) (ADVP (RB up) (PP (TO to) (NP (DT the) (NN plate)))) (S (VP (TO to) (VP (VB support) (NP (DT the) (JJ beleaguered) (NN floor) (NNS traders)) (PP (IN by) (S (VP (VBG buying) (NP (NP (JJ big) (NNS blocks)) (PP (IN of) (NP (NN stock))))))))))))))) (, ,) (NP (NNS traders)) (VP (VBP say)) (. .))"
ts2 = "(SINV (S (ADVP (RB Once) (RB again)) (-LRB- -LCB-) (NP (DT the) (NNS specialists)) (-RRB- -RCB-) (VP (VBD were) (RB not) (ADJP (JJ able) (S (VP (TO to) (VP (VB handle) (NP (NP (DT the) (NNS imbalances)) (PP (IN on) (NP (NP (DT the) (NN floor)) (PP (IN of) (NP (DT the) (NNP New) (NNP York) (NNP Stock) (NNP Exchange)))))))))))) (, ,) ('' '') (VP (VBD said)) (NP (NP (NNP Christopher) (NNP Pedersen)) (, ,) (NP (NP (JJ senior) (NN vice) (NN president)) (PP (IN at) (NP (NNP Twenty-First) (NNP Securities) (NNP Corp))))) (. .))"
ts3 = "(S (NP (NP (NNP Seven) (NNP Big) (NNP Board) (NNS stocks)) (: --) (NP (NP (NNP UAL)) (, ,) (NP (NNP AMR)) (, ,) (NP (NNP BankAmerica)) (, ,) (NP (NNP Walt) (NNP Disney)) (, ,) (NP (NNP Capital) (NNP Cities\/ABC)) (, ,) (NP (NNP Philip) (NNP Morris)) (CC and) (NP (NNP Pacific) (NNP Telesis) (NNP Group))) (: --)) (VP (VP (VBD stopped) (S (VP (NN trading)))) (CC and) (VP (ADVP (RB never)) (VBD resumed))) (. .))"
ts4 = "(S (S (NP (JJ Big) (NN investment) (NNS banks)) (VP (VBD refused) (S (VP (TO to) (VP (VB step) (ADVP (RB up) (PP (TO to) (NP (DT the) (NN plate)))) (S (VP (TO to) (VP (VB support) (NP (DT the) (JJ beleaguered) (NN floor) (NNS traders)) (PP (IN by) (S (VP (VBG buying) (NP (NP (JJ big) (NNS blocks)) (PP (IN of) (NP (NN stock))))))))))))))) (, ,) (NP (NNS traders)) (VP (VBP say)) (. .))"
ts5 = "(FRAG (SBAR (IN As) (IN in) (: :) (`` ``) (SQ (NP (PRP You)) (VP (VBD went) (NP (VBG ballooning))) (. ?) (. ?) (. !) (. !))))"
ts6 = "(NP (NP (NN Year)) (VP (VBN ended) (NP (NNP Dec.) (CD 31) (, ,) (CD 1988))) (X (NN \*)))"
ts7 = "(S (INTJ (RB No)) (, ,) (NP (PRP it)) (VP (VBD was) (RB n't) (NP (NNP Black) (NNP Monday))) (. .))"
ts8 = "(S (NP (DT The) (JJ finger-pointing)) (VP (VBZ has) (ADVP (RB already)) (VP (VBN begun))) (. .))"
ts9 = "(S (`` ``) (NP (DT The) (NN equity) (NN market)) (VP (VBD was) (ADJP (JJ illiquid))) (. .))"
ts10 = "(S (PP (IN In) (NP (NP (DT an) (NNP Oct.) (CD 19) (NN review)) (PP (IN of) (NP (`` ``) (NP (DT The) (NN Misanthrope)) ('' '') (PP (IN at) (NP (NP (NNP Chicago) (POS 's)) (NNP Goodman) (NNP Theatre))))) (PRN (-LRB- -LRB-) (`` ``) (S (NP (JJ Revitalized) (NNS Classics)) (VP (VBP Take) (NP (DT the) (NNP Stage)) (PP (IN in) (NP (NNP Windy) (NNP City))))) (, ,) ('' '') (NP (NNP Leisure) (CC &) (NNP Arts)) (-RRB- -RRB-)))) (, ,) (NP (NP (NP (DT the) (NN role)) (PP (IN of) (NP (NNP Celimene)))) (, ,) (VP (VBN played) (PP (IN by) (NP (NNP Kim) (NNP Cattrall)))) (, ,)) (VP (VBD was) (VP (ADVP (RB mistakenly)) (VBN attributed) (PP (TO to) (NP (NNP Christina) (NNP Haag))))) (. .))"

## Encoding playground

In [59]:
ts = ts10
debug=True
tree = C_Tree.from_string(ts)
tree = tree.collapse_unary()
tree = C_Tree.to_binary_right(tree)
pt(debug, tree)

                                                                                                                                                                                 S                                                                                                                                                                              
                   ______________________________________________________________________________________________________________________________________________________________|__________________________________________________________________________________________________                                                                             
                  PP                                                                                                                                                                                                                                                                |                

In [66]:
# GAP OPERATOR
def insert_children_on_empty_child(tree, child):
    if tree.children[0].label == "EMPTY":
        tree.children[0] = child
    else:
        tree.children[1] = child
    return tree
def generate_leaf_tree(postag, word):
    return C_Tree(label=postag, children=C_Tree(label=word))

def operator_1(gap_stack, stack, word, last_common):
    # > Create new sub-tree with label = last_common, left_child=Tree(postag, word), right_child=None
    new_tree = C_Tree(label=last_common, children=[generate_leaf_tree(postag="P", word=word), C_Tree(label="EMPTY")])
    # > Push new sub-tree to stack
    stack.append(new_tree)
    return gap_stack, stack

def operator_open_gap(gap_stack, stack, word, last_common):
    # > Create new sub-tree with label = last_common, left_child=Tree(postag, word), right_child=None and push to stack
    new_tree = C_Tree(label=last_common, children=[generate_leaf_tree(postag="P", word=word), C_Tree(label="EMPTY")])
    stack.append(new_tree)

    # > Push new sub-tree to open_gaps_stack
    gap_stack.append(new_tree)
    return gap_stack, stack


def operator_n1(gap_stack, stack, word, last_common):
    # > Create new sub-tree with label = last_common, left_child=popped_tree, right_child=None and push to stack    
    stack_top = stack.pop()
    new_children = generate_leaf_tree(postag="P", word=word)
    insert_children_on_empty_child(stack_top, new_children)
    
    # > Pop stack and insert a Tree(postag, word) in the empty child of the popped tree
    new_tree = C_Tree(label=last_common, children=[stack_top, C_Tree(label="EMPTY")])
    stack.append(new_tree)

    return gap_stack, stack

def operator_close_gap(gap_stack, stack, word, last_common):
    # > Pop stack and insert a Tree(postag, word) in the empty child of the popped tree
    stack_top = stack.pop()
    new_children = generate_leaf_tree(postag="P", word=word)
    insert_children_on_empty_child(stack_top, new_children)
    
    # If we have more than open gaps, we need to find the last open gap that is still in the stack
    last_open_gap_position_index = 0
    if len(gap_stack) > 1:
        last_open_gap = gap_stack.pop()    
        while last_open_gap not in stack and len(gap_stack) > 0:
            last_open_gap = gap_stack.pop()

        last_open_gap_position_index = stack.index(last_open_gap) if last_open_gap in stack else 0
    
    # > Pop stack until we reach the last open gap
    #       => this loop will run at least once
    #       => this loop will merge all elements in the stack from the top until the 'last open gap'
    for i in range(last_open_gap_position_index, len(stack)):
        new_top = stack.pop()
        insert_children_on_empty_child(new_top, stack_top)
        stack_top = new_top

    # > Create new sub-tree with label = last_common, left_child=popped_tree, right_child=None and push to stack
    new_tree = C_Tree(label=last_common, children=[stack_top, C_Tree(label="EMPTY")])
    stack.append(new_tree)

    return gap_stack, stack

ts = ts4
debug = False
encoder = C_NaiveRelativeEncoding(separator="_",
                                    unary_joiner="++",
                                    reverse=False,
                                    binary=True,
                                    binary_direction="R",
                                    binary_marker="[*]")

ptb_path = "/home/poli/Documents/Treebanks/PENN_TREEBANK/train.trees"
# open
ptb_file = open(ptb_path, "r")
# read
for line in ptb_file:
    tree = C_Tree.from_string(line.rstrip())
    lc_tree = encoder.encode(tree)

    new_labels = []
    for label in lc_tree.labels:
        new_nc = ""
        if lc_tree.labels.index(label)==0:
            new_nc = "G"
        elif lc_tree.labels.index(label)==len(lc_tree.labels)-1:
            new_nc = "-G"
        elif label.n_commons == 1:
            new_nc = "1"
        elif label.n_commons == -1:
            new_nc = "-1"
        elif label.n_commons < 0:
            new_nc = "-G"
        else:
            new_nc = "G"
        lc = label.last_common if lc_tree.labels.index(label)!=len(lc_tree.labels)-1 else "$$"
        new_labels.append((new_nc, lc))




    gap_stack = []
    stack     = []

    for label, word in zip(new_labels, lc_tree.words):
        if label[0] == "1":
            gap_stack, stack = operator_1(gap_stack, stack, word, label[1])
        elif label[0] == "-1":
            gap_stack, stack = operator_n1(gap_stack, stack, word, label[1])
        elif label[0] == "G":
            gap_stack, stack = operator_open_gap(gap_stack, stack, word, label[1])
        elif label[0] == "-G":
            gap_stack, stack = operator_close_gap(gap_stack, stack, word, label[1])
        else:
            raise ValueError("Unknown label: {}".format(label))

    result_tree = stack[0].children[0]
    result_tree = C_Tree.restore_from_binary()

    print(str(result_tree),'\n',line.rstrip())

(S[*] (NP[*] (NP (S (PRN[*] (PRN[*] (PP (P In) (NP (NP (P an) (NP[*] (P Oct.) (NP[*] (P 19) (P review)))) (NP[*] (PP (P of) (NP (P ``) (NP[*] (NP (P The) (P Misanthrope)) (NP[*] (P '') (PP (P at) (NP (NP (P Chicago) (P 's)) (NP[*] (P Goodman) (P Theatre)))))))) (PRN (P -LRB-) (PRN[*] (P ``) (S (NP (P Revitalized) (P Classics)) (VP (P Take) (VP[*] (NP (P the) (P Stage)) (PP (P in) (NP (P Windy) (P City))))))))))) (PRN[*] (P ,) (PRN[*] (P '') (NP (P Leisure) (NP[*] (P &) (P Arts)))))) (P -RRB-)) (S[*] (P ,) (NP (NP (P the) (P role)) (PP (P of) (P Celimene))))) (NP[*] (P ,) (VP (P played) (PP (P by) (NP (P Kim) (P Cattrall)))))) (P ,)) (S[*] (VP (P was) (VP (P mistakenly) (VP[*] (P attributed) (PP (P to) (NP (P Christina) (P Haag)))))) (P .))) 
 S (PP (IN In) (NP (NP (DT an) (NNP Oct.) (CD 19) (NN review)) (PP (IN of) (NP (`` ``) (NP (DT The) (NN Misanthrope)) ('' '') (PP (IN at) (NP (NP (NNP Chicago) (POS 's)) (NNP Goodman) (NNP Theatre))))) (PRN (-LRB- -LRB-) (`` ``) (S (NP (JJ Revitali

KeyboardInterrupt: 

In [3]:
all_ts = [tsb]
debug = True
encoder = C_NaiveRelativeEncoding(separator="_",
                                    unary_joiner="++",
                                    reverse=False,
                                    binary=True,
                                    binary_direction="R",
                                    binary_marker="*")
for ts in all_ts:
    pt(debug, ts)
    
    tree = C_Tree.from_string(ts)
    lc_tree = encoder.encode(tree)

    if debug:
        print(lc_tree)

    dec_tree = encoder.decode(lc_tree)
    dec_tree = dec_tree.postprocess_tree(conflict_strat=C_STRAT_MAX, clean_nulls=True)
    
    pt(debug, dec_tree)
    
    assert str(dec_tree) == ts

        NTA                                            
  _______|_______________                               
 |                      NTB                            
 |            ___________|_______________               
 |          NTC                         NTG            
 |    _______|___                     ___|___________   
 |   |          NTF                 NTH              | 
 |   |        ___|_______     _______|___            |  
 |   |      NTD          |   |          NTI          | 
 |   |    ___|___        |   |    _______|___        |  
 |   |   |      NTE      |   |   |          NTJ      | 
 |   |   |    ___|___    |   |   |        ___|___    |  
 PA  PB  PC  PD      PE  PF  PG  PH      PI      PJ  PK
 |   |   |   |       |   |   |   |       |       |   |  
 a   b   c   d       e   f   g   h       i       j   k 

-BOS-	-BOS-	-BOS-
a	PA	1*_NTA
b	PB	2*_NTC
c	PC	2*_NTD
d	PD	1*_NTE
e	PE	-2*_NTF
f	PF	-2*_NTB
g	PG	2*_NTH
h	PH	1*_NTI
i	PI	1*_NTJ
j	PJ	-3*_NTG
k	PK	-2*_NT

## Labels equivalence

In [4]:
ptb_path = "~/Treebanks/const/PENN_TREEBANK/"
ptb_path = os.path.expanduser(ptb_path)
print("Reading PTB from",ptb_path)    
df = pd.DataFrame(columns=["Word", "Relative", "Tetratag"])
with open(os.path.join(ptb_path,"test.trees")) as f:
    ptb = f.read().splitlines()

for tree_string in ptb:
    tree = C_Tree.from_string(tree_string)
    rel_encoder = C_NaiveRelativeEncoding(separator="_", unary_joiner="+", reverse=False, binary=True, binary_marker="[*]")
    tet_encoder = C_Tetratag(separator="_", unary_joiner="+", reverse=False, binary_marker="[*]")

    t_lin_tree = tet_encoder.encode(copy.deepcopy(tree))
    r_lin_tree = rel_encoder.encode(copy.deepcopy(tree))

    df_temp = pd.DataFrame(columns=["Word", "Relative", "Tetratag"])
    
    df_temp["Word"] = r_lin_tree.words
    df_temp["Relative"] = [str(l.n_commons) for l in r_lin_tree.labels]
    df_temp["Tetratag"] = [l.n_commons for l in t_lin_tree.labels]
    
    df = pd.concat([df, df_temp], axis=0)

    
    r_decoded_tree = rel_encoder.decode(r_lin_tree)
    r_decoded_tree = r_decoded_tree.postprocess_tree(conflict_strat=C_STRAT_MAX, clean_nulls=True)
    
    t_decoded_tree = tet_encoder.decode(t_lin_tree)
    t_decoded_tree = t_decoded_tree.postprocess_tree(conflict_strat=C_STRAT_MAX, clean_nulls=True)
        
    if not (str(r_decoded_tree)==tree_string):
        print("Decoded tree is different from original")
        print("Original:",tree_string)
        print("Decoded:",r_decoded_tree)

Reading PTB from /home/poli/Treebanks/const/PENN_TREEBANK/


FileNotFoundError: [Errno 2] No such file or directory: '/home/poli/Treebanks/const/PENN_TREEBANK/test.trees'

In [ ]:
# print relative values for each of the unique values of tetratag
tetra_values = df["Tetratag"].unique()
tetra_values.sort()
for tetratag in tetra_values:
    print("Tetratag:",tetratag,\
        "\n\t==> Relative:",df[df["Tetratag"]==tetratag]["Relative"].unique())

Tetratag: l 
	==> Relative: ['-3' '-4' '-1' '-2' '-5' '0' '-6' '-7' '-12' '-9' '-14' '-8' '-10' '-11']
Tetratag: lL 
	==> Relative: ['-3' '-4' '-1' '-5' '-13' '-6' '-2' '-11' '-8' '-14' '-7' '-10' '-12'
 '-9' '-19' '-16' '-15' '-17' '-27' '-22' '-20' '-18' '-25' '-21' '-23'
 '-30' '-26']
Tetratag: lR 
	==> Relative: ['-4' '-6' '-5' '-3' '-2' '-13' '-1' '-15' '-7' '-9' '-11' '-8' '-17'
 '-10' '-14' '-12' '-16' '-19']
Tetratag: r 
	==> Relative: ['1']
Tetratag: rL 
	==> Relative: ['1' '2' '0' '3']
Tetratag: rR 
	==> Relative: ['1' '2' '3' '4' '5' '6']
